# Notebook 00: Image Preprocessing Pipeline (Layer 0)

This notebook demonstrates the 6-stage image preprocessing pipeline that normalizes
and enhances PDF page images before they enter any downstream pipeline.

**Stages:**
1. Resolution Normalization
2. Deskew Correction
3. Non-Local Means Denoising
4. CLAHE Contrast Enhancement
5. Border & Artifact Removal
6. Adaptive Binarization (OCR path only)

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import fitz  # PyMuPDF
from pathlib import Path

from src.preprocessing.image_preprocessor import ImagePreprocessor

# Load config
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

preprocessor = ImagePreprocessor(config)
print('Preprocessor loaded with config:', {k: getattr(preprocessor, k) for k in ['target_dpi', 'deskew_threshold_deg', 'denoise_h', 'clahe_clip', 'binarize_method']})

## Load a Sample PDF

Place your test PDF in `notebooks/sample_pdfs/` and update the path below.

In [ ]:
# Update this path to your test PDF
PDF_PATH = 'sample_pdfs/v1.pdf'
PAGE_NUMBER = 0  # 0-indexed

# Load and render page to image
doc = fitz.open(PDF_PATH)
page = doc[PAGE_NUMBER]
print(f'Document: {PDF_PATH}')
print(f'Total pages: {doc.page_count}')
print(f'Page {PAGE_NUMBER} size: {page.rect.width:.0f} x {page.rect.height:.0f} points')

# Render at 300 DPI
pix = page.get_pixmap(dpi=300)
raw_image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
if pix.n == 4:  # RGBA -> BGR
    raw_image = cv2.cvtColor(raw_image, cv2.COLOR_RGBA2BGR)
elif pix.n == 3:  # RGB -> BGR
    raw_image = cv2.cvtColor(raw_image, cv2.COLOR_RGB2BGR)

print(f'Image shape: {raw_image.shape}, dtype: {raw_image.dtype}')

plt.figure(figsize=(12, 16))
plt.imshow(cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB))
plt.title(f'Raw Page {PAGE_NUMBER} @ 300 DPI')
plt.axis('off')
plt.show()

## Stage-by-Stage Preprocessing

Let's run each stage individually and visualize the before/after.

In [ ]:
def show_before_after(before, after, title_before, title_after, figsize=(20, 10)):
    """Display two images side-by-side for comparison."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    if len(before.shape) == 3:
        ax1.imshow(cv2.cvtColor(before, cv2.COLOR_BGR2RGB))
    else:
        ax1.imshow(before, cmap='gray')
    ax1.set_title(title_before)
    ax1.axis('off')
    
    if len(after.shape) == 3:
        ax2.imshow(cv2.cvtColor(after, cv2.COLOR_BGR2RGB))
    else:
        ax2.imshow(after, cmap='gray')
    ax2.set_title(title_after)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Stage 2: Deskew Correction
deskewed, angle = preprocessor.deskew(raw_image.copy())
print(f'Detected skew angle: {angle:.3f} degrees')
show_before_after(raw_image, deskewed, 'Original', f'After Deskew ({angle:.2f} deg)')

In [ ]:
# Stage 3: Denoising
denoised = preprocessor.denoise(deskewed.copy())
print(f'Denoised image shape: {denoised.shape}')

# Zoom into a region to see denoising effect
h, w = deskewed.shape[:2]
crop_y, crop_x = h // 4, w // 4
crop_h, crop_w = h // 4, w // 4

if len(deskewed.shape) == 3:
    before_crop = deskewed[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]
else:
    before_crop = deskewed[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]

if len(denoised.shape) == 3:
    after_crop = denoised[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]
else:
    after_crop = denoised[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]

show_before_after(before_crop, after_crop, 'Before Denoise (zoomed)', 'After Denoise (zoomed)')

In [ ]:
# Stage 4: CLAHE Contrast Enhancement
enhanced = preprocessor.enhance_contrast(denoised.copy())
show_before_after(denoised, enhanced, 'Before CLAHE', 'After CLAHE')

In [ ]:
# Stage 5: Border Removal
no_border = preprocessor.remove_borders(enhanced.copy())
print(f'Before border removal: {enhanced.shape}')
print(f'After border removal: {no_border.shape}')
show_before_after(enhanced, no_border, 'Before Border Removal', 'After Border Removal')

In [ ]:
# Stage 6: Adaptive Binarization (for OCR)
binarized = preprocessor.adaptive_binarize(no_border.copy())
print(f'Binarized shape: {binarized.shape} (method: {preprocessor.binarize_method})')
show_before_after(no_border, binarized, 'Before Binarization', f'After {preprocessor.binarize_method.title()} Binarization')

## Full Pipeline Run

Run the complete preprocessing pipeline in one call.

In [ ]:
# Full pipeline: visual comparison path (no binarization)
processed_visual = preprocessor.process(raw_image.copy(), for_ocr=False)
print(f'Visual path output: {processed_visual.shape}')
show_before_after(raw_image, processed_visual, 'Raw Input', 'After Full Pipeline (Visual Path)')

In [ ]:
# Full pipeline: OCR path (with binarization)
processed_ocr = preprocessor.process(raw_image.copy(), for_ocr=True)
print(f'OCR path output: {processed_ocr.shape}')
show_before_after(raw_image, processed_ocr, 'Raw Input', 'After Full Pipeline (OCR Path)')

## Histogram Analysis

Compare pixel intensity distributions before and after preprocessing.

In [ ]:
def to_gray(img):
    if len(img.shape) == 3:
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, img, title in [
    (axes[0], to_gray(raw_image), 'Raw Image'),
    (axes[1], to_gray(processed_visual), 'After Pipeline (Visual)'),
    (axes[2], processed_ocr if len(processed_ocr.shape) == 2 else to_gray(processed_ocr), 'After Pipeline (OCR)'),
]:
    ax.hist(img.ravel(), 256, [0, 256], color='steelblue', alpha=0.7)
    ax.set_title(f'{title}\nmean={img.mean():.1f}, std={img.std():.1f}')
    ax.set_xlim([0, 256])

plt.tight_layout()
plt.show()

In [ ]:
doc.close()
print('Done! Preprocessing pipeline validated.')